# Classification with Logistic Regression

# Objectives

- Recognize the model-less baseline understanding for a classification task
- Use logistic regression to perform a classification task

## Set Up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For our modeling steps
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Your task: predict who will survive the sinking of the Titanic, given the passenger manifest. So we are predicting the column `survived`

Dataset details: https://www.kaggle.com/c/titanic/data

In [ ]:
# Getting the data from sklearn
dfX, dfy = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
# Cleaning a bit to get to a full dataframe of the data
df = dfX.copy()
df = df.drop(columns=['boat', 'body', 'home.dest'])
df['survived'] = dfy

df.head()

In [ ]:
df.info()

## Model-Less Baseline

As always, we want to know what we're up against: How hard is our problem? Do we even need to use a model here?

Can we think of a strategy, without modeling, to guess whether a person survives the Titanic?

- 


In [ ]:
# Implement the strategy here!


In [ ]:
# Create our predictions as something list-like


In [ ]:
# Use sklearn's accuracy_score to compare actual to predictions


## Modeling with Logistic Regression!

Let's build our first model! What features should we use?

In [ ]:
# Explore the data to decide which columns to start with

In [ ]:
# Define your initial X and y

In [ ]:
# Train test split

### Preprocessing

Don't forget - SKLearn's implementation of Logistic Regression penalizes coefficients by default - at the very least, we'll need to scale our features!

If we want to use any categorical or object-type columns as input variables, we'll need to encode them. We will also need to deal with any null values in columns we want to use.

[A really useful resource/example of processing the Titanic data!](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html)

In [ ]:
# Implement preprocessing steps here


## Actual Model!

Now that we've preprocessed our data, let's model it!

In [ ]:
# Instantiate our logistic regression model

# Fit our model


In [ ]:
# Evaluate: use .score to check accuracy on train and test


## Interpreting Coefficients

In [ ]:
# Check out our coefficients - what do we learn?


## Iterating to Optimize Hyperparameters

We've built an initial model, and now there are a lot of ways we can potentially improve our models. For example, we could use more features, drop less useful features, engineer smarter features, etc.

BUT one thing we haven't explored so far is optimizing hyperparameters! Specifically, we might think about optimizing how strong our regularization parameter - should we penalize our model more? less? 

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# Let's use cross_validate to try out different regularization strengths
# I envision this as a for loop, looping over different options for C 
# Don't forget to capture the outputs of each loop somewhere!



In [ ]:
# Explore our results


In [ ]:
# Build a new model with the best regularization strength, on the full train set


In [ ]:
# Then evaluate on train and test


#### Discuss!

- 


----

## Level Up: Notes on Cost Functions, and Solutions To the Optimization Problem

Unlike the least-squares problem for linear regression, no one has yet found a closed-form solution to the optimization problem presented by logistic regression. But even if one exists, the computation would no doubt be so complex that we'd be better off using some sort of approximation method instead.

But there's still a problem.

Recall the cost function for linear regression: <br/><br/>
$SSE = \Sigma_i(y_i - \hat{y}_i)^2 = \Sigma_i(y_i - (\beta_0 + \beta_1x_{i1} + ... + \beta_nx_{in}))^2$.

This function, $SSE(\vec{\beta})$, is convex.

If we plug in our new logistic equation for $\hat{y}$, we get: <br/><br/>
$SSE_{log} = \Sigma_i(y_i - \hat{y}_i)^2 = \Sigma_i\left(y_i - \left(\frac{1}{1+e^{-(\beta_0 + \beta_1x_{i1} + ... + \beta_nx_{in})}}\right)\right)^2$.

### The Bad News

*This* function, $SSE_{log}(\vec{\beta})$, is [**not** convex](https://towardsdatascience.com/why-not-mse-as-a-loss-function-for-logistic-regression-589816b5e03c).

That means that, if we tried to use gradient descent or some other approximation method that looks for the minimum of this function, we could easily find a local rather than a global minimum.

> Note that the scikit-learn class *expects the user to specify the solver* to be used in calculating the coefficients. The default solver, [lbfgs](https://en.wikipedia.org/wiki/Limited-memory_BFGS), works well for many applications.

### The Good News

We can use **log-loss** instead:

$\mathcal{L}(\vec{y}, \hat{\vec{y}}) = -\frac{1}{N}\Sigma^N_{i=1}\left(y_iln(\hat{y}_i)+(1-y_i)ln(1-\hat{y}_i)\right)$,

where $\hat{y}_i$ is the probability that $(x_{i1}, ... , x_{in})$ belongs to **class 1**.

**More resources on the log-loss function**:

https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11

https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a

### 🧠 Knowledge Check

- Is a bigger value (more positive) better or worse than a smaller value?

- What would the log-loss for one data point when the target is $0$ but we predict $1$?

- What would the log-loss for one data point when the target is $0$ but we predict $0$?

## Level Up: More on Interpreting Logistic Regression Coefficients

In [ ]:
logreg.coef_

How do we interpret the coefficients of a logistic regression? For a linear regression, the situaton was like this:

- Linear Regression: We construct the best-fit line and get a set of coefficients. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change in $y$.

- Logistic Regression: We find the coefficients of the best-fit line by some approximation method. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change (not in $y$ but) in $ln\left(\frac{y}{1-y}\right)$.

We have:

$\ln\left(\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}\right) = \ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k$.

Exponentiating both sides:

$\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)} = e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k}$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right)}\cdot e^k$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^k\cdot\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}$

That is, the odds ratio at $x_1+1$ has increased by a factor of $e^k$ relative to the odds ratio at $x_1$.

For more on interpretation, see [this page](https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/how-to/binary-logistic-regression/interpret-the-results/all-statistics-and-graphs/coefficients/).

## Level UP:  Other Link Functions, Other Models

Logistic regression's link function is the logit function, but different sorts of models use different link functions.

[Wikipedia](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) has a nice table of generalized linear model types and their associated link functions.